In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd

from datascience import *
import numpy as np

import locale
import re
import csv

### Initial Analysis of Data Set

- Which departments get the most funding? 
- What type of research do they conduct? 
- Where is the money coming from, by department?

In [2]:
grant_data = Table.read_table('grant_data.csv')

In [3]:
# Cleaning the initial data set

def replace_empty_dept(department, division):
    if department == 'nan':
        dept_div = division
    else:
        dept_div = department
    replace = ['School of ', 'DO NOT USE ', 'College of ', 'Department ']
    for word in replace:
        dept_div = dept_div.replace(word, '')
    return dept_div

def int_monetary_value(money_str):
    money_str = money_str.replace('$', '').replace(' ', '').replace(',', '')
    if '(' in money_str:
        money_str = '-' + money_str.replace('(', '').replace(')', '')
    return int(money_str)

def extract_year(date):
    return int(date[-4:])

grant_data.append_column('Dept/Division', grant_data.apply(replace_empty_dept, ['Department', 'Division']))
grant_data.append_column('Grant Amount', grant_data.apply(int_monetary_value, 'Amount'))
grant_data.append_column('Year', grant_data.apply(extract_year, 'Project Begin Date'))

grant_data

Activity Type,Amount,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title,Dept/Division,Grant Amount,Year
Applied research,"$179,032",State of California,California Department of Health Care Services,School of Public Health,nan,15952,021331-002,"Colford Jr, John M",7/1/2006,9/30/2006,DNS AIDS Training,Public Health,179032,2006
Basic research,"$154,578",State of California,California Department of Social Services,School of Social Welfare,nan,15959,021362-002,"Needell, Barbara",7/1/2006,9/30/2006,Performance Indicators/California Children's Services Ar ...,Social Welfare,154578,2006
Instruction,"$225,000",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,Social Welfare,225000,2006
Instruction,"$47,138",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,Social Welfare,47138,2006
Basic research,"$65,000",Federal,NIH National Institutes of Health - Miscellaneous,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),78561,021425-002,"Keasling, Jay",7/1/2006,12/31/2006,Model-Driven Strain Engineering for Isoprenoid Drug Prod ...,The California Institute for Quantitative Biosciences (QB3),65000,2006
Other,"$30,937",Not for Profit,E.J. Slosson Endowment Fund,VC Research Museums & Field Stations,Botanical Garden,91478,020907-002,"Carmichael, A Christopher",7/1/2006,6/30/2007,UC Botanical Garden Interpretive Project,Botanical Garden,30937,2006
Other,$0,Not for Profit,E.J. Slosson Endowment Fund,VC Research Museums & Field Stations,Botanical Garden,91478,020907-002,"Carmichael, A Christopher",7/1/2006,6/30/2007,UC Botanical Garden Interpretive Project,Botanical Garden,0,2006
Basic research,"$55,474",Federal,NSF National Science Foundation,Division of Physical Sciences,Berkeley Seismological Laboratory,11089,021063-002,"Nadeau, Robert",7/1/2006,6/30/2007,Seismic Indicators of Time-Varying Deep Deformation Alon ...,Berkeley Seismological Laboratory,55474,2006
Basic research,"$27,000",Not for Profit,California Walnut Board (Frmly Walnut Marketing Board),CNR College of Natural Resources,"Environmental Science, Policy & Management",78527,021120-002,"Welter, Stephen C",7/1/2006,6/30/2007,Evaluation of Alternative Pheromone Dispensing Technologies,"Environmental Science, Policy & Management",27000,2006
Basic research,"$50,000",University of California,UC Biotechnology Research and Education (BREP),Division of Physical Sciences,Physics,nan,021264-002,"Liphardt, Jan T",7/1/2006,6/30/2007,Model System to Study Translocation of Single Biopolymer ...,Physics,50000,2006


In [4]:
grant_data.sort('Grant Amount', descending=True).to_df().to_csv('cleaned_research_spo_data.csv', index=False)

In [5]:
# Analyzing by Department

by_dept = grant_data.select([0, 2, 'Dept/Division', 'Grant Amount', 'Year']).group('Dept/Division', list)
by_dept.append_column('Grant Amount', by_dept.apply(sum, 'Grant Amount list'))
by_dept = by_dept.drop('Grant Amount list')
by_dept

Dept/Division,Activity Type list,Sponsor Class list,Year list,Grant Amount
ARE Agricultural & Resource Economics,"['Basic research', 'Basic research', 'Basic research', ' ...","['State of California', 'Federal', 'Federal', 'Federal', ...","[2005, 2006, 2004, 2002, 2006, 2006, 2004, 2006, 2007, 2 ...",5543973
ARF Archaeological Research Facility,"['Basic research', 'Basic research', 'Basic research', ' ...","['Federal', 'University of California', 'Federal', 'Fede ...","[2006, 2006, 2007, 2006, 2007, 2007, 2007, 2008, 2007, 2 ...",1926295
Academic Achievement Division,"['Other', 'Services', 'Services', 'Other', 'Training', ' ...","['Federal', 'Federal', 'Federal', 'Federal', 'Federal', ...","[2007, 2010, 2010, 2007, 2012, 2010, 2012, 2010, 2010, 2015]",8391223
Anthropology,"['Basic research', 'Basic research', 'Applied research', ...","['University of California', 'Federal', 'Federal', 'Fede ...","[2005, 2006, 2006, 2005, 2006, 2006, 2007, 2005, 2007, 2 ...",1963652
Architecture,"['Other', 'Basic research', 'Basic research', 'Other', ' ...","['State of California', 'Industry', 'University of Calif ...","[2004, 2006, 2007, 2004, 2005, 2007, 2005, 2004, 2004, 2 ...",202026
Art History,"['Basic research', 'Basic research', 'Basic research', ' ...","['Not for Profit', 'Not for Profit', 'Not for Profit', ' ...","[2007, 2007, 2007, 2008, 2008, 2008, 2009, 2008, 2012, 2 ...",911662
Art Mus & Pacific Film Archive,"['Other', 'Basic research', 'Other', 'Other', 'Other', ' ...","['Federal', 'Federal', 'Not for Profit', 'Non Federal Go ...","[2006, 2004, 2005, 2005, 2006, 2006, 2005, 2006, 2005, 2 ...",5178877
Art Practice,"['Basic research', 'Basic research']","['Not for Profit', 'Not for Profit']","[2008, 2008]",238000
Associate VC - Business & Administrative Services,"['Other', 'Other', 'Other']","['Non Federal Governmental', 'Non Federal Governmental', ...","[2004, 2005, 2005]",0
Associate VC - Health & Human Services,"['Other', 'Basic research', 'Other', 'Other', 'Other', ' ...","['Non Federal Governmental', 'Federal', 'Not for Profit' ...","[2005, 2006, 2006, 2006, 2005, 2001, 2005, 2003, 2006, 2 ...",4050302


In [6]:
# Reformatting currency

def format_currency(grant):
    return '${:,}'.format(grant)

def format_money_table(table):
    table = table.sort('Grant Amount', descending=True)
    table.append_column('Grant Amount', table.apply(format_currency, 'Grant Amount'))
    return table

In [7]:
# Exploring sources of funding for the top grant recipients

def sponsor_counts(sponsor_list):
    count_list = [sponsor_list.count('Federal'), sponsor_list.count('State of California'), sponsor_list.count('Not for Profit'),\
            sponsor_list.count('University of California')]
    count_list.append(len(sponsor_list) - sum(count_list))
    return count_list

counts = Table(['Federal', 'State of California', 'Non Profit', 'University of California', 'Other']).with_rows(
                    by_dept.apply(sponsor_counts, 'Sponsor Class list'))

by_dept_funding = by_dept.drop([1,2,3]).with_columns(['Federal', counts[0],
                                                      'State of California', counts[1],
                                                      'Non Profit', counts[2],
                                                      'University of California', counts[3],
                                                      'Other', counts[4]])

by_dept_funding = format_money_table(by_dept_funding)
by_dept_funding

by_dept_funding.to_csv('by_dept_funding.csv')

In [8]:
# Exploring what kind of research is taking place for each grant recipient
def activity_counts(activity_list):
    count_list = [activity_list.count('Applied research'), activity_list.count('Basic research'), activity_list.count('Training'), \
                  activity_list.count('Instruction'), activity_list.count('Services'), activity_list.count('Other')]
    count_list[5] += len(activity_list) - sum(count_list)
    return count_list

counts = Table(['Applied research', 'Basic research', 'Training', 'Instruction', 'Services', 'Other']).with_rows(
                    by_dept.apply(activity_counts, 'Activity Type list'))

by_dept_activity = by_dept.drop([1,2,3]).with_columns(['Applied Research', counts[0],
                                                      'Basic Research', counts[1],
                                                      'Training', counts[2],
                                                      'Instruction', counts[3],
                                                      'Services', counts[4],
                                                      'Other', counts[5]])
total = counts[0]
for i in np.arange(1, 6):
    total += counts[i]

by_dept_activity = by_dept_activity.with_column('Total', total)
by_dept_activity = format_money_table(by_dept_activity)
by_dept_activity

by_dept_activity.to_csv('by_dept_activity.csv')

### Interactive Data Exploration

- How much have different sources invested in research and how much are they emphasizing it?
- Which departments are most involved in heavily funded research? 

In [9]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [10]:
def year(table, yr, show_all_entries):
    table = table.where('Year', yr)
    if show_all_entries:
        table.show()
        return
    return table

def department(table, dept, show_all_entries):
    table = table.where('Dept/Division', dept)
    if show_all_entries:
        table.show()
        return
    return table

def amount(table, threshold_amount, show_all_entries):
    table = table.where(table['Grant Amount'] > threshold_amount).sort('Grant Amount', descending=True)
    if show_all_entries:
        table.show()
        return
    return table

trends = grant_data.drop([1] + list(range(4, 11)))

In [13]:
interact(year, table=fixed(trends), yr=widgets.IntSlider(min=1997,max=2016,step=1,value=2005), show_all_entries=False)

Activity Type,Sponsor Class,Sponsor,Title,Dept/Division,Grant Amount,Year
Basic research,Not for Profit,Urban Creeks Council of California,An Investigation of the Potential of Eucalyptus Wood as ...,Geography,0,2005
Basic research,Federal,NSF National Science Foundation,Collaborative Research: Ice and Rock Processes in Cirques,Geography,0,2005
Basic research,Federal,NIH National Institute of General Medical Sciences,Heme Protein Sensors in Prokaryotes,The California Institute for Quantitative Biosciences (QB3),110016,2005
Applied research,State of California,California Department of Industrial Relations,Impact of the New California Permanent Disability Rating ...,ISSI Institute for the Study of Societal Issues,0,2005
Applied research,Not for Profit,Shell Foundation,Promotion of Technology Innovation and Dissemination for ...,Public Health,41215,2005
Other,Not for Profit,National Film Preservation Foundation,Preservation of Films by Dion Vigne and Edward Silversto ...,Art Mus & Pacific Film Archive,0,2005
Basic research,Federal,USDI Geological Survey,Heat Flux Modeling for the Sacramento-San Joaquin Delta,Civil & Environmental Engineering,-166,2005
Basic research,Industry,Fanuc Ltd,Intelligent Control of Robot Manipulator for Performance ...,ERSO Engineering Research Support Organization,161488,2005
Basic research,University of California,UC California HIV/AIDS Research Program,HIV Risk Research and Homeless Sexual Minority Youth,Anthropology,0,2005
Basic research,Federal,NASA National Aeronautics and Space Administration Godda ...,Assessing and Implementing High Reliability at NASA Thro ...,IBER Institute of Business & Economic Research,0,2005


<function __main__.year>

In [14]:
interact(department, table=fixed(trends), dept=list(set(trends['Dept/Division'])), show_all_entries=False)

Activity Type,Sponsor Class,Sponsor,Title,Dept/Division,Grant Amount,Year
Other,Not for Profit,Walter & Elise Haas Fund,Richmond/Berkeley Project,Consortium of the Arts,35000,2006
Other,Not for Profit,Walter & Elise Haas Fund,Richmond/Berkeley Project,Consortium of the Arts,25000,2006


<function __main__.department>

In [13]:
interact(amount, table=fixed(trends), \
         threshold_amount=widgets.IntSlider(min=0,max=max(trends['Grant Amount'])-1,step=10000, value=0), show_all_entries=False)

Activity Type,Sponsor Class,Sponsor,Title,Dept/Division,Grant Amount,Year
Basic research,Industry,"BP Group (BP America, BP Exploration, British Petroleum)",Energy Biosciences Institute,Vice Chancellor for Research ORUs,346371180,2007
Basic research,Federal,NASA National Aeronautics and Space Administration Godda ...,The Ionospheric CONnection Explorer (ICON) Phases B-F,SSL Space Sciences Lab,132635047,2012
Basic research,Not for Profit,Simons Foundation,Simons Institute for the Theory of Computing,Simons Institute for the Theory of Computing,59640000,2012


In [14]:
top_100_grants = trends.sort('Grant Amount', descending=True).take(range(0,100))
top_100_grants = top_100_grants.with_column('Grant Ranking', np.arange(1, 101))
top_100_grants

Activity Type,Sponsor Class,Sponsor,Title,Dept/Division,Grant Amount,Year,Grant Ranking
Basic research,Industry,"BP Group (BP America, BP Exploration, British Petroleum)",Energy Biosciences Institute,Vice Chancellor for Research ORUs,346371180,2007,1
Basic research,Federal,NASA National Aeronautics and Space Administration Godda ...,The Ionospheric CONnection Explorer (ICON) Phases B-F,SSL Space Sciences Lab,132635047,2012,2
Basic research,Not for Profit,Simons Foundation,Simons Institute for the Theory of Computing,Simons Institute for the Theory of Computing,59640000,2012,3
Instruction,State of California,California Department of Social Services,Title IV-E Social Work Training Program,Social Welfare,32745069,2010,4
Instruction,State of California,California Department of Social Services,Title IV-E Social Work Training Program,Social Welfare,32587458,2012,5
Instruction,State of California,California Department of Social Services,Title IV-E Social Work Training Progam,Social Welfare,32463656,2011,6
Instruction,State of California,California Department of Social Services,Title IV-E Social Work Training Program,Social Welfare,30752572,2009,7
Basic research,Federal,NSF National Science Foundation,Developing the Preliminary Design for the Deep Undergrou ...,DUSEL Deep Underground Science & Engineering Laboratory,29092000,2009,8
Instruction,State of California,California Department of Social Services,Title IVE Social Work Training Program,Social Welfare,28986867,2008,9
Basic research,Not for Profit,MARCO Microelectronics Advanced Research Corporation,The TerraSwarm Research Center,ERSO Engineering Research Support Organization,27573125,2013,10
